In [2]:
import sqlite3
import pandas as pd
import numpy as np

# WPP2024_POP_F02_1_POPULATION_5-YEAR_AGE_GROUPS_BOTH_SEXES.xlsx

In [17]:
dd = pd.read_excel('DataDict.xlsx', sheet_name = 'F02')

file = 'WPP2024_POP_F02_1_POPULATION_5-YEAR_AGE_GROUPS_BOTH_SEXES.xlsx'
country = pd.read_excel('Country.xlsx')
for i in dd.index:
    t = dd.loc[i,'Tab']
    r = dd.loc[i,'rank']
    v = dd.loc[i, 'Variable']
    if dd.isnull().loc[i,'Formula']:
        f = dd.loc[i,'Formula']
    else:
        f = "val" + str(dd.loc[i,'Formula'])
    if dd.isnull().loc[i,'Decimal Places']:
        d = dd.loc[i,'Decimal Places']
    else:
        d = int(dd.loc[i,'Decimal Places'])
    if r == 1:
        if t == 'Estimates':
            df = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = t
                    )
            col = ['Region, subregion, country or area *','Year',
                    '15-19',
                    '20-24',
                    '25-29',
                    ]
            df = df[col]
            df = df.set_index(['Region, subregion, country or area *','Year',])
            df = df.apply(pd.to_numeric, errors='coerce')
            df['val'] = df.sum(axis = 1)
            df = df.reset_index()

            if dd.isnull().loc[i,'Formula']:
                df = df
            else:
                df.val = df.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                df = df
            else:
                df.val = round(df.val, d)
            df.Year = df.Year.astype('Int64')
            
            n = 'Region, subregion, country or area *'
            y = 'Year'
            first_last = df.sort_values([n,y]).groupby(df[n])['val'] \
                    .agg(['first', 'last']).reset_index()
            first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
            df = df.pivot_table(index = n,
                            columns= y,
                            values = 'val').reset_index()
            df = df.merge(first_last,
                        on = n,
                        how = 'left')
            df = country.merge(df,
                            on = n,
                            how = 'left')
            df = df.drop(columns= n)
            df = df.drop_duplicates()
            df.to_csv(f'IFs Import/{v}.csv',
                    index=False
                    )
        else:
            df = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = 'Estimates'
                    )
            col = ['Region, subregion, country or area *','Year',
                    '15-19',
                    '20-24',
                    '25-29',
                    ]
            df = df[col]
            df = df.set_index(['Region, subregion, country or area *','Year',])
            df = df.apply(pd.to_numeric, errors='coerce')
            df['val'] = df.sum(axis = 1)
            df = df.reset_index()

            if dd.isnull().loc[i,'Formula']:
                df = df
            else:
                df.val = df.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                df = df
            else:
                df.val = round(df.val, d)
            df.Year = df.Year.astype('Int64')

            dt = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = t
                    )
            col = ['Region, subregion, country or area *','Year',
                    '15-19',
                    '20-24',
                    '25-29',
                    ]
            dt = dt[col]
            dt = dt.set_index(['Region, subregion, country or area *','Year',])
            dt = dt.apply(pd.to_numeric, errors='coerce')
            dt['val'] = dt.sum(axis = 1)
            dt = dt.reset_index()

            if dd.isnull().loc[i,'Formula']:
                dt = dt
            else:
                dt.val = dt.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                dt = dt
            else:
                dt.val = round(dt.val, d)
            dt.Year = dt.Year.astype('Int64')

            final = pd.concat([df,dt], ignore_index = True)
            n = 'Region, subregion, country or area *'
            y = 'Year'
            first_last = final.sort_values([n,y]).groupby(final[n])['val'] \
                    .agg(['first', 'last']).reset_index()
            first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
            final = final.pivot_table(index = n,
                            columns= y,
                            values = 'val').reset_index()
            final = final.merge(first_last,
                        on = n,
                        how = 'left')
            final = country.merge(final,
                            on = n,
                            how = 'left')
            final = final.drop(columns= n)
            final = final.drop_duplicates()
            final.to_csv(f'IFs Import/{v}.csv',
                     index=False
                     )
    if r == 2:
        if t == 'Estimates':
            df = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = t
                    )
            df = df.set_index(['Region, subregion, country or area *','Year',])
            df = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8,]], axis=1)
            df = df.apply(pd.to_numeric, errors='coerce')
            df['total'] = df.sum(axis = 1)
            df['15-29'] = df[[    
                        '15-19',
                        '20-24',
                        '25-29',
                        ]].sum(axis = 1)
            df['0-14'] = df[[
                        '0-4',
                        '5-9',
                        '10-14'
                        ]].sum(axis = 1)
            df['val'] = df['15-29']/(df['total']-df['0-14'])
            df = df.reset_index()
            

            if dd.isnull().loc[i,'Formula']:
                df = df
            else:
                df.val = df.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                df = df
            else:
                df.val = round(df.val, d)
            df.Year = df.Year.astype('Int64')
            
            n = 'Region, subregion, country or area *'
            y = 'Year'
            first_last = df.sort_values([n,y]).groupby(df[n])['val'] \
                    .agg(['first', 'last']).reset_index()
            first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
            df = df.pivot_table(index = n,
                            columns= y,
                            values = 'val').reset_index()
            df = df.merge(first_last,
                        on = n,
                        how = 'left')
            df = country.merge(df,
                            on = n,
                            how = 'left')
            df = df.drop(columns= n)
            df = df.drop_duplicates()
            df.to_csv(f'IFs Import/{v}.csv',
                    index=False
                    )
        else:
            df = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = 'Estimates'
                    )
            df = df.set_index(['Region, subregion, country or area *','Year',])
            df = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8,]], axis=1)
            df = df.apply(pd.to_numeric, errors='coerce')
            df['total'] = df.sum(axis = 1)
            df['15-29'] = df[[    
                        '15-19',
                        '20-24',
                        '25-29',
                        ]].sum(axis = 1)
            df['0-14'] = df[[
                        '0-4',
                        '5-9',
                        '10-14'
                        ]].sum(axis = 1)
            df['val'] = df['15-29']/(df['total']-df['0-14'])
            df = df.reset_index()

            if dd.isnull().loc[i,'Formula']:
                df = df
            else:
                df.val = df.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                df = df
            else:
                df.val = round(df.val, d)
            df.Year = df.Year.astype('Int64')

            dt = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = t
                    )
            dt = dt.set_index(['Region, subregion, country or area *','Year',])
            dt = dt.drop(dt.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8,]], axis=1)
            dt = dt.apply(pd.to_numeric, errors='coerce')
            dt['total'] = dt.sum(axis = 1)
            dt['15-29'] = dt[[    
                        '15-19',
                        '20-24',
                        '25-29',
                        ]].sum(axis = 1)
            dt['0-14'] = dt[[
                        '0-4',
                        '5-9',
                        '10-14'
                        ]].sum(axis = 1)
            dt['val'] = dt['15-29']/(dt['total']-dt['0-14'])
            dt = dt.reset_index()

            if dd.isnull().loc[i,'Formula']:
                dt = dt
            else:
                dt.val = dt.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                dt = dt
            else:
                dt.val = round(dt.val, d)
            dt.Year = dt.Year.astype('Int64')

            final = pd.concat([df,dt], ignore_index = True)
            n = 'Region, subregion, country or area *'
            y = 'Year'
            first_last = final.sort_values([n,y]).groupby(final[n])['val'] \
                    .agg(['first', 'last']).reset_index()
            first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
            final = final.pivot_table(index = n,
                            columns= y,
                            values = 'val').reset_index()
            final = final.merge(first_last,
                        on = n,
                        how = 'left')
            final = country.merge(final,
                            on = n,
                            how = 'left')
            final = final.drop(columns= n)
            final = final.drop_duplicates()
            final.to_csv(f'IFs Import/{v}.csv',
                     index=False
                     )
    if r == 3:
        if t == 'Estimates':
            df = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = t
                    )
            df = df.set_index(['Region, subregion, country or area *','Year',])
            df = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8,]], axis=1)
            df = df.apply(pd.to_numeric, errors='coerce')
            df['total'] = df.sum(axis = 1)
            df['15-29'] = df[[    
                        '15-19',
                        '20-24',
                        '25-29',
                        ]].sum(axis = 1)
            df['0-29'] = df[[
                        '0-4',
                        '5-9',
                        '10-14',
                        '15-29'
                        ]].sum(axis = 1)
            df['val'] = df['15-29']/(df['total']-df['0-29'])
            df = df.reset_index()
            

            if dd.isnull().loc[i,'Formula']:
                df = df
            else:
                df.val = df.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                df = df
            else:
                df.val = round(df.val, d)
            df.Year = df.Year.astype('Int64')
            
            n = 'Region, subregion, country or area *'
            y = 'Year'
            first_last = df.sort_values([n,y]).groupby(df[n])['val'] \
                    .agg(['first', 'last']).reset_index()
            first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
            df = df.pivot_table(index = n,
                            columns= y,
                            values = 'val').reset_index()
            df = df.merge(first_last,
                        on = n,
                        how = 'left')
            df = country.merge(df,
                            on = n,
                            how = 'left')
            df = df.drop(columns= n)
            df = df.drop_duplicates()
            df.to_csv(f'IFs Import/{v}.csv',
                    index=False
                    )
        else:
            df = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = 'Estimates'
                    )
            df = df.set_index(['Region, subregion, country or area *','Year',])
            df = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8,]], axis=1)
            df = df.apply(pd.to_numeric, errors='coerce')
            df['total'] = df.sum(axis = 1)
            df['15-29'] = df[[    
                        '15-19',
                        '20-24',
                        '25-29',
                        ]].sum(axis = 1)
            df['0-29'] = df[[
                        '0-4',
                        '5-9',
                        '10-14',
                        '15-29'
                        ]].sum(axis = 1)
            df['val'] = df['15-29']/(df['total']-df['0-29'])
            df = df.reset_index()

            if dd.isnull().loc[i,'Formula']:
                df = df
            else:
                df.val = df.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                df = df
            else:
                df.val = round(df.val, d)
            df.Year = df.Year.astype('Int64')

            dt = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = t
                    )
            dt = dt.set_index(['Region, subregion, country or area *','Year',])
            dt = dt.drop(dt.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8,]], axis=1)
            dt = dt.apply(pd.to_numeric, errors='coerce')
            dt['total'] = dt.sum(axis = 1)
            dt['15-29'] = dt[[    
                        '15-19',
                        '20-24',
                        '25-29',
                        ]].sum(axis = 1)
            dt['0-29'] = dt[[
                        '0-4',
                        '5-9',
                        '10-14',
                        '15-29'
                        ]].sum(axis = 1)
            dt['val'] = dt['15-29']/(dt['total']-dt['0-29'])
            dt = dt.reset_index()

            if dd.isnull().loc[i,'Formula']:
                dt = dt
            else:
                dt.val = dt.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                dt = dt
            else:
                dt.val = round(dt.val, d)
            dt.Year = dt.Year.astype('Int64')

            final = pd.concat([df,dt], ignore_index = True)
            n = 'Region, subregion, country or area *'
            y = 'Year'
            first_last = final.sort_values([n,y]).groupby(final[n])['val'] \
                    .agg(['first', 'last']).reset_index()
            first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
            final = final.pivot_table(index = n,
                            columns= y,
                            values = 'val').reset_index()
            final = final.merge(first_last,
                        on = n,
                        how = 'left')
            final = country.merge(final,
                            on = n,
                            how = 'left')
            final = final.drop(columns= n)
            final = final.drop_duplicates()
            final.to_csv(f'IFs Import/{v}.csv',
                     index=False
                     )
    else:
        if t == 'Estimates':
            df = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = t
                    )
            df = df.set_index(['Region, subregion, country or area *','Year',])
            df = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8,]], axis=1)
            df = df.apply(pd.to_numeric, errors='coerce')
            df['total'] = df.sum(axis = 1)
            df['15-29'] = df[[    
                        '15-19',
                        '20-24',
                        '25-29',
                        ]].sum(axis = 1)
            df['val'] = df['15-29']/df['total']
            df = df.reset_index()
            

            if dd.isnull().loc[i,'Formula']:
                df = df
            else:
                df.val = df.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                df = df
            else:
                df.val = round(df.val, d)
            df.Year = df.Year.astype('Int64')
            
            n = 'Region, subregion, country or area *'
            y = 'Year'
            first_last = df.sort_values([n,y]).groupby(df[n])['val'] \
                    .agg(['first', 'last']).reset_index()
            first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
            df = df.pivot_table(index = n,
                            columns= y,
                            values = 'val').reset_index()
            df = df.merge(first_last,
                        on = n,
                        how = 'left')
            df = country.merge(df,
                            on = n,
                            how = 'left')
            df = df.drop(columns= n)
            df = df.drop_duplicates()
            df.to_csv(f'IFs Import/{v}.csv',
                    index=False
                    )
        else:
            df = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = 'Estimates'
                    )
            df = df.set_index(['Region, subregion, country or area *','Year',])
            df = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8,]], axis=1)
            df = df.apply(pd.to_numeric, errors='coerce')
            df['total'] = df.sum(axis = 1)
            df['15-29'] = df[[    
                        '15-19',
                        '20-24',
                        '25-29',
                        ]].sum(axis = 1)
            df['val'] = df['15-29']/df['total']
            df = df.reset_index()

            if dd.isnull().loc[i,'Formula']:
                df = df
            else:
                df.val = df.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                df = df
            else:
                df.val = round(df.val, d)
            df.Year = df.Year.astype('Int64')

            dt = pd.read_excel(f'{file}',
                    skiprows = 16,
                    sheet_name = t
                    )
            dt = dt.set_index(['Region, subregion, country or area *','Year',])
            dt = dt.drop(dt.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8,]], axis=1)
            dt = dt.apply(pd.to_numeric, errors='coerce')
            dt['total'] = dt.sum(axis = 1)
            dt['15-29'] = dt[[    
                        '15-19',
                        '20-24',
                        '25-29',
                        ]].sum(axis = 1)
            dt['val'] = dt['15-29']/dt['total']
            dt = dt.reset_index()

            if dd.isnull().loc[i,'Formula']:
                dt = dt
            else:
                dt.val = dt.eval(f)
            if dd.isnull().loc[i,'Decimal Places']:
                dt = dt
            else:
                dt.val = round(dt.val, d)
            dt.Year = dt.Year.astype('Int64')

            final = pd.concat([df,dt], ignore_index = True)
            n = 'Region, subregion, country or area *'
            y = 'Year'
            first_last = final.sort_values([n,y]).groupby(final[n])['val'] \
                    .agg(['first', 'last']).reset_index()
            first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
            final = final.pivot_table(index = n,
                            columns= y,
                            values = 'val').reset_index()
            final = final.merge(first_last,
                        on = n,
                        how = 'left')
            final = country.merge(final,
                            on = n,
                            how = 'left')
            final = final.drop(columns= n)
            final = final.drop_duplicates()
            final.to_csv(f'IFs Import/{v}.csv',
                     index=False
                     )
            
# df
# final

# CSV

In [ ]:
dd = pd.read_excel('DataDict.xlsx', sheet_name = 'CSV')
# dd = dd[dd['Column'] == 'Median Age, as of 1 July (years)']
file = 'WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_FULL.xlsx'
csv = 'WPP2024_Demographic_Indicators_OtherVariants.csv/WPP2024_Demographic_Indicators_OtherVariants.csv'
country = pd.read_excel('Country.xlsx')
country_csv = pd.read_excel('Country_CSV.xlsx')

dt = pd.read_csv(f'{csv}',
            encoding='windows-1252'
               )
dt = dt[dt['Variant'] == 'Constant mortality']
dt = dt[dt['LocTypeName'] == 'Country/Area']
col = [
'Location',
'Time',
'TPopulation1July',
'TPopulationMale1July',
'TPopulationFemale1July',
'MedianAgePop',
'Births',
'CBR',
'TFR',
'CDR',
'LEx',
'LExMale',
'LExFemale',
'IMR',
'Under5Deaths',
'NetMigrations',
'CNMR',
]
dt_all = dt[col]


for i in dd.index:
    t = dd.loc[i,'Tab']
    c = dd.loc[i,'Column']
    c_csv = dd.loc[i,'Column_CSV']
    v = dd.loc[i, 'Variable']
    if dd.isnull().loc[i,'Formula']:
        f = dd.loc[i,'Formula']
    else:
        f = "val" + str(dd.loc[i,'Formula'])
    if dd.isnull().loc[i,'Decimal Places']:
        d = dd.loc[i,'Decimal Places']
    else:
        d = int(dd.loc[i,'Decimal Places'])
    df = pd.read_excel(f'{file}',
               skiprows = 16,
               sheet_name = 'Estimates'
               )
    col = ['Region, subregion, country or area *','Year', c ]
    df = df[col]
    df.rename(columns={f'{c}': 'val'}, inplace=True)
    df.val = df.val.apply(pd.to_numeric, errors='coerce')
    if dd.isnull().loc[i,'Formula']:
        df = df
    else:
        df.val = df.eval(f)
    if dd.isnull().loc[i,'Decimal Places']:
        df = df
    else:
        df.val = round(df.val, d)
    df.Year = df.Year.astype('Int64')
    df = country.merge(df, on = 'Region, subregion, country or area *', how = 'left')
    df = df.drop(columns='Region, subregion, country or area *')
    df = df.drop(columns='FIPS_CODE')
    
    
    col = ['Location','Time', c_csv ]
    dt = dt_all[col]
    # dt = dt
    dt.rename(columns={f'{c_csv}': 'val',
                       'Time':'Year'
                       }, inplace=True)
    dt.val = dt.val.apply(pd.to_numeric, errors='coerce')
    if dd.isnull().loc[i,'Formula']:
        dt = dt
    else:
        dt.val = dt.eval(f)
    if dd.isnull().loc[i,'Decimal Places']:
        dt = dt
    else:
        dt.val = round(dt.val, d)
    dt.Year = dt.Year.astype('Int64')
    dt = country_csv.merge(dt, on = 'Location', how = 'left')
    dt = dt.drop(columns='Location')
    dt = dt.drop(columns='FIPS_CODE')
    
    final = pd.concat([df,dt], ignore_index=True)
    n = 'Country'
    y = 'Year'
    first_last = final.sort_values([n,y]).groupby(final[n])['val'] \
            .agg(['first', 'last']).reset_index()
    first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
    final = final.pivot_table(index = n,
                    columns= y,
                    values = 'val').reset_index()
    final = final.merge(first_last,
                on = n,
                how = 'left')
    final = country.merge(final,
                    on = n,
                    how = 'left')
    final = final.drop(columns= 'Region, subregion, country or area *')
    final = final.drop_duplicates()
    final.to_csv(f'IFs Import/{v}.csv',
             index=False
             ) 

# Sheet1

In [ ]:
dd = pd.read_excel('DataDict.xlsx')
country = pd.read_excel('Country.xlsx')
for i in dd.index:
    t = dd.loc[i,'Tab']
    c = dd.loc[i,'Column']
    v = dd.loc[i, 'Variable']
    file = dd.loc[i,'File']
    if dd.isnull().loc[i,'Formula']:
        f = dd.loc[i,'Formula']
    else:
        f = "val" + str(dd.loc[i,'Formula'])
    if dd.isnull().loc[i,'Decimal Places']:
        d = dd.loc[i,'Decimal Places']
    else:
        d = int(dd.loc[i,'Decimal Places'])
    if t == 'Estimates':
        df = pd.read_excel(f'{file}',
                   skiprows = 16,
                   sheet_name = 'Estimates'
                   )
        col = ['Region, subregion, country or area *','Year', c ]
        df = df[col]
        df.rename(columns={f'{c}': 'val'}, inplace=True)
        df.val = df.val.apply(pd.to_numeric, errors='coerce')
        if dd.isnull().loc[i,'Formula']:
            df = df
        else:
            df.val = df.eval(f)
        if dd.isnull().loc[i,'Decimal Places']:
            df = df
        else:
            df.val = round(df.val, d)
        df.Year = df.Year.astype('Int64')
        n = 'Region, subregion, country or area *'
        y = 'Year'
        first_last = df.sort_values([n,y]).groupby(df[n])['val'] \
                .agg(['first', 'last']).reset_index()
        first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
        df = df.pivot_table(index = n,
                        columns= y,
                        values = 'val').reset_index()
        df = df.merge(first_last,
                    on = n,
                    how = 'left')
        df = country.merge(df,
                        on = n,
                        how = 'left')
        df = df.drop(columns= n)
        df = df.drop_duplicates()
        df.to_csv(f'IFs Import/{v}.csv',
                 index=False
                 )
    else:
        df = pd.read_excel(f'{file}',
                   skiprows = 16,
                   sheet_name = 'Estimates'
                   )
        col = ['Region, subregion, country or area *','Year', c ]
        df = df[col]
        df.rename(columns={f'{c}': 'val'}, inplace=True)
        df.val = df.val.apply(pd.to_numeric, errors='coerce')
        if dd.isnull().loc[i,'Formula']:
            df = df
        else:
            df.val = df.eval(f)
        if dd.isnull().loc[i,'Decimal Places']:
            df = df
        else:
            df.val = round(df.val, d)
        df.Year = df.Year.astype('Int64')
        
        dt = pd.read_excel(f'{file}',
                   skiprows = 16,
                   sheet_name = t
                   )
        col = ['Region, subregion, country or area *','Year', c ]
        dt = dt[col]
        dt.rename(columns={f'{c}': 'val'}, inplace=True)
        dt.val = dt.val.apply(pd.to_numeric, errors='coerce')
        if dd.isnull().loc[i,'Formula']:
            dt = dt
        else:
            dt.val = dt.eval(f)
        if dd.isnull().loc[i,'Decimal Places']:
            dt = dt
        else:
            dt.val = round(dt.val, d)
        dt.Year = dt.Year.astype('Int64')

        final = pd.concat([df,dt], ignore_index=True)
        n = 'Region, subregion, country or area *'
        y = 'Year'
        first_last = final.sort_values([n,y]).groupby(final[n])['val'] \
                .agg(['first', 'last']).reset_index()
        first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
        final = final.pivot_table(index = n,
                        columns= y,
                        values = 'val').reset_index()
        final = final.merge(first_last,
                    on = n,
                    how = 'left')
        final = country.merge(final,
                        on = n,
                        how = 'left')
        final = final.drop(columns= n)
        final = final.drop_duplicates()
        final.to_csv(f'IFs Import/{v}.csv',
                 index=False
                 )